# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание проекта**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину. 

Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.


**Условия задачи:**

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.


## Загрузка и подготовка данных

In [38]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np



<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>

Данные геологоразведки трёх регионов:

In [39]:
try:
    df_geo_data_0 = pd.read_csv('geo_data_0.csv')
    df_geo_data_1 = pd.read_csv('geo_data_1.csv')
    df_geo_data_2 = pd.read_csv('geo_data_2.csv')
except:
    df_geo_data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    df_geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    df_geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

**Описание данных:**

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Анализ датафреймов:

In [40]:
df_geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [41]:
df_geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [42]:
df_geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Типы данных везде корректные

In [43]:
df_geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [44]:
df_geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [45]:
df_geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [46]:
df_geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [47]:
df_geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [48]:
df_geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [49]:
list_of_dataframes = [df_geo_data_0, df_geo_data_1, df_geo_data_2]

In [50]:
for i in list_of_dataframes:
    print(i.shape)

(100000, 5)
(100000, 5)
(100000, 5)


Размерность датафреймов совпадает

In [51]:
df_geo_data_0.isna().sum().sort_values()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [52]:
df_geo_data_1.isna().sum().sort_values()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [53]:
df_geo_data_2.isna().sum().sort_values()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Пропущенных значений в датафреймах нет

Проверка на дубликаты:

In [54]:
df0 = df_geo_data_0['id'].duplicated().sum()
df0

10

In [55]:
df1 = df_geo_data_1['id'].duplicated().sum()
df1

4

In [56]:
df2 = df_geo_data_2['id'].duplicated().sum()
df2

4

In [57]:
df_geo_data_0['id'].drop_duplicates()
df_geo_data_0.duplicated().sum()

0

In [58]:
df_geo_data_1['id'].drop_duplicates()
df_geo_data_1.duplicated().sum()

0

In [59]:
df_geo_data_2['id'].drop_duplicates()
df_geo_data_2.duplicated().sum()

0

Промежуточный вывод:

- Датафреймы исследованы
- Пропусков не выявлено
- Дубликаты в поле id удалены
- Выбросы/аномалии не выявлены

product — объём запасов в скважине (тыс. баррелей).

In [60]:
# Prepare Data
x0 = df_geo_data_0['product'].values.tolist()
x1 = df_geo_data_1['product'].values.tolist()
x2 = df_geo_data_2['product'].values.tolist()

# Draw Plot
pyplot.figure(figsize=(16, 10), dpi= 1000)
sns.distplot(x0, color="g", label="0", hist_kws={'alpha':.7}, kde_kws={'linewidth':3})
sns.distplot(x1, color="deeppink", label="1", hist_kws={'alpha':.7}, kde_kws={'linewidth':3})
sns.distplot(x2, color="dodgerblue", label="2", hist_kws={'alpha':.7}, kde_kws={'linewidth':3})

# Decoration
pyplot.title('Плотность распределения product', fontsize=22)
pyplot.legend()
pyplot.show()

/opt/conda/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function 

Грифик плотности распределения *product* — объём запасов в скважине (тыс. баррелей).


## Обучение и проверка модели

In [61]:
train_0, valid_0 = train_test_split(df_geo_data_0, test_size=0.25, random_state=12345)
features_train_0 = train_0.drop(['product', 'id'], axis=1)
target_train_0 = train_0['product']

features_valid_0 = valid_0.drop(['product', 'id'], axis=1)
target_valid_0 = valid_0['product']

In [62]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predicted_valid_0 = model.predict(features_valid_0)
rmse = sqrt(mean_squared_error(target_valid_0, predicted_valid_0))

print(f"RMSE = {rmse}")
print(f"Средний запас предсказанного сырья = {predicted_valid_0.mean()}")

RMSE = 37.5794217150813
Средний запас предсказанного сырья = 92.59256778438035


In [63]:
train_1, valid_1 = train_test_split(df_geo_data_1, test_size=0.25, random_state=12345)
features_train_1 = train_1.drop(['product', 'id'], axis=1)
target_train_1 = train_1['product']

features_valid_1 = valid_1.drop(['product', 'id'], axis=1)
target_valid_1 = valid_1['product'] 

In [64]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_valid_1 = model.predict(features_valid_1)
rmse = sqrt(mean_squared_error(target_valid_1, predicted_valid_1))

print(f"RMSE = {rmse}")
print(f"Средний запас предсказанного сырья = {predicted_valid_1.mean()}")

RMSE = 0.893099286775617
Средний запас предсказанного сырья = 68.728546895446


In [65]:
train_2, valid_2 = train_test_split(df_geo_data_2, test_size=0.25, random_state=12345)
features_train_2 = train_2.drop(['product', 'id'], axis=1)
target_train_2 = train_2['product']

features_valid_2 = valid_2.drop(['product', 'id'], axis=1)
target_valid_2 = valid_2['product'] 

In [66]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predicted_valid2 = model.predict(features_valid_2)
rmse = sqrt(mean_squared_error(target_valid_2, predicted_valid2))

print(f"RMSE = {rmse}")
print(f"Средний запас предсказанного сырья = {predicted_valid2.mean()}")

RMSE = 40.02970873393434
Средний запас предсказанного сырья = 94.96504596800489


Наибольший средний запас сырья в 3 регионе - 94.9, но также и высокое RMSE. Во втором регионе RMSE близка к 1.

- Средние запас для 1 региона: 92.5
- Средние запас для 2 региона: 68.7
- Средние запас для 3 региона: 94.9

## Подготовка к расчёту прибыли

In [67]:
investigated_points = 500
best_of_investigated_points = 200
budget = 10000000000
price_per_barrel = 450000
risk_threshold = 0.025
bootstrap_samples = 1000

In [68]:
# Делим весь бюджет на количество точек - получаем затрачиваемый бюджет на одну точку
# далее делим на единицу барреля
single_point_budget = budget / best_of_investigated_points
print('Затрачиваемый бюджет на одну точку:', single_point_budget)

Затрачиваемый бюджет на одну точку: 50000000.0


**Расчёт объёма сырья нужного для безыубыточного состояния**

In [69]:
breakeven = single_point_budget / price_per_barrel
print('Достаточный объём сырья для безубыточной разработки новой скважины:', breakeven)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


**Изучение средних запасов сырья в каждом регионе**

In [70]:
for i in list_of_dataframes:
    print(i['product'].mean())

92.50000000000001
68.82500000000002
95.00000000000004


Средниие запасы сырья в 1 регионе: 92.5

Средниие запасы сырья в 2 регионе: 68.8

Средниие запасы сырья в 3 регионе: 95.0

Промежуточный вывод:

Так как **Достаточный объём сырья для безубыточной разработки новой скважины - 111.1** можем сделать вывод возможной убыточности всез трех регионов

В первом и третьем регионе средние значения 92.5 и 95 соответсвтенно, во втором - 68.8, что совсем мало.

Количество скважин с запасами более 111.1 в 1 и 3 регионе немало, следовательно и окупиться они могут при правильном выборе скважин



## Расчёт прибыли и рисков 

**Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:**

In [71]:
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    product = selected.sum()
    revenue = product * price_per_barrel
    final_profit = revenue - budget
    return final_profit

Бутстреп для каждого региона;

Вычисление средней прибыли;

Вычисление 95% доверительного интервала;

Вычисление риска убытка

In [72]:
state = np.random.RandomState(12345) 

values_0 = []
lost_0 = 0
target_valid0 = target_valid_0.reset_index(drop=True)
for i in range(1000):
    target_subsample_0 = target_valid0.sample(n=500, replace=True, random_state=state)
    valid_subsample_0 = pd.Series(predicted_valid_0)[target_subsample_0.index]
    profit0 = profit(target_subsample_0, valid_subsample_0, 200)
    if profit0 < 0:
            lost_0 +=1
    values_0.append(profit0)

values0 = pd.Series(values_0)
lower0 = values0.quantile(0.025)
upper0 = values0.quantile(0.975)
mean0 = values0.mean()
risk0 = lost_0 / 1000

print(f"Средняя прибыль: {mean0}")
print(f"95%-ый доверительный интервал: от {lower0} до {upper0}")
print('Процент риска {:.2%}'.format(risk0))

Средняя прибыль: 425938526.91059244
95%-ый доверительный интервал: от -102090094.83793654 до 947976353.3583689
Процент риска 6.00%


In [73]:
state = np.random.RandomState(12345) 

values_1 = []
lost_1 = 0
target_valid1 = target_valid_1.reset_index(drop=True)
for i in range(1000):
    target_subsample_1 = target_valid1.sample(n=500, replace=True, random_state=state)
    valid_subsample_1 = pd.Series(predicted_valid_1)[target_subsample_1.index]
    profit_1 = profit(target_subsample_1, valid_subsample_1, 200)
    if profit_1 < 0:
            lost_1 +=1
    values_1.append(profit_1)

values_1 = pd.Series(values_1)
mean_1 = values_1.mean()

lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
risk_1 = lost_1 / 1000

print("Средняя прибыль:", mean_1)
print(f"95%-ый доверительный интервал: от {lower_1} до {upper_1}")
print('Процент риска {:.2%}'.format(risk_1))

Средняя прибыль: 515222773.4432899
95%-ый доверительный интервал: от 68873225.37050176 до 931547591.2570494
Процент риска 1.00%


In [74]:
state = np.random.RandomState(12345)

values_2 = []
lost_2 = 0
target_valid_2 = target_valid_2.reset_index(drop=True)
for i in range(1000):
    target_subsample_2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    valid_subsample_2 = pd.Series(predicted_valid2)[target_subsample_2.index]
    profit_2 = profit(target_subsample_2, valid_subsample_2, 200)
    if profit_2 < 0:
            lost_2 +=1
    values_2.append(profit_2)

values_2 = pd.Series(values_2)
mean_2 = values_2.mean()

lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
risk_2 = lost_2 / 1000

print("Средняя прибыль:", mean_2)
print(f"95%-ый доверительный интервал: от {lower_2} до {upper_2}")
print('Процент риска {:.2%}'.format(risk_2))

Средняя прибыль: 435008362.7827556
95%-ый доверительный интервал: от -128880547.32978901 до 969706954.1802661
Процент риска 6.40%


**Общий вывод:**

Были иследованны 3 датафрейма: на пропуски, на дубликаты, размерность и описательную статистику

- Созданы модели и сделаны предсказания на валидационной выборке
- Ключевые значения, нужные для расчетов были глобально сохранены
- Рассчитан достаточный объём сырья для безубыточной разработки новой скважины.
- Сравнен со средним запасом в каждом регионе.
- Написана функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.
- Посчитаны риски и прибыль для каждого региона.
Для этого была применена техника Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
Вычислен 95%-й доверительный интервал и риск убытков.

Самый прибыльный регион - второй

- Средняя прибыль: 515222773.4
- Риск появления убытка - 1.00%

**Следовательно бурить новую скважину следует во втором регионе.**